# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Условия задачи:**

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
* Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

Загрузите и подготовьте данные. Поясните порядок действий.

Импортируем необходимые библиотеки и прочитаем файлы в переменные

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np

Сохраним в константу количество представленных датафреймов

In [2]:
N = 3 #количество датафреймов

Создадим список и добавим в него каждый датафрейм как отдельный элемент

In [3]:
df_list = []

df = pd.DataFrame()

for i in range(N):
    df = pd.read_csv(f'/datasets/geo_data_{i}.csv')
    df_list.append(df)

Посмотрим общую информацию о данных

In [4]:
for df in df_list:
    print(df.info(), '\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Colu

Посмотрим первые строки файлов

In [5]:
df_list[0].head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [6]:
df_list[1].head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
df_list[2].head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Ничего не понятно, но очень интересно

Проверим пропуски и дубликаты в данных

In [8]:
for df in df_list:
    print(df.isna().sum(), '\n')

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 



In [9]:
for df in df_list:
    print(df.duplicated().sum(), '\n')

0 

0 

0 



Посмотрим на данные в целевом столбце

In [10]:
for df in df_list:
    print(df['product'].value_counts().count(), '\n')

100000 

12 

100000 



Очень странно, во втором датафрейме всего 12 уникальный значений

## Обучение и проверка модели

Разбейте данные на обучающую и валидационную выборки в соотношении 75:25

In [11]:
features_list = []
target_list = []
features_train_list = []
features_valid_list = []
target_train_list = []
target_valid_list = []

In [12]:
for df in df_list:
    features = df.drop(['product', 'id'], axis=1) 
    target = df['product']

    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, train_size=0.75, random_state=12345)
    
    features_list.append(features)
    target_list.append(target)
    features_train_list.append(features_train)
    target_train_list.append(target_train)
    features_valid_list.append(features_valid)
    target_valid_list.append(target_valid)

Обучите модель и сделайте предсказания на валидационной выборке

In [13]:
model_list = []

for i in range(N):
    model = LinearRegression().fit(features_train_list[i], target_train_list[i])
    model_list.append(model)

In [14]:
predictions_list = []

for i in range(N):
    predictions = model_list[i].predict(features_valid_list[i])
    predictions_list.append(predictions)

Сохраните предсказания и правильные ответы на валидационной выборке.

In [15]:
common_list = []

for i in range (N):
    common = pd.DataFrame()
    common['predictions'] = pd.Series(predictions_list[i])
    common['target'] = target_valid_list[i].reset_index(drop=True)
    common_list.append(common)

Напечатайте на экране средний запас предсказанного сырья и RMSE модели

In [16]:
for common in common_list:
    print(common.mean(), '\n')

predictions    92.592568
target         92.078597
dtype: float64 

predictions    68.728547
target         68.723136
dtype: float64 

predictions    94.965046
target         94.884233
dtype: float64 



In [17]:
for i in range(N):
    print(sqrt(mean_squared_error(target_valid_list[i], predictions_list[i])))

37.5794217150813
0.893099286775617
40.02970873393434


Проанализируйте результаты

Модели 1 и 3 регионов имеют высокую среднеквадратичную ошибку, но у модели 2 региона она мала, при этом в исходном датафрейме у 2 региона всего 12 уникальных значений в целевом признаке  
Средние значения предсказаний и целевого признака при этом близки  
Средние значения запасов в регионах 1 и 3 выше, чем в регионе 2
У 1 региона меньше среднеквадратичная ошибка, но средний запас тоже меньше, у 3 региона - наоборот

## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.  
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.  
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.  

Все ключевые значения для расчётов сохраните в отдельных переменных

In [18]:
BUDGET = 10e9
TOTAL_OIL_WELLS = 500
BEST_OIL_WELLS = 200
BARREL_PROFIT = 450
PRODUCT_PROFIT = BARREL_PROFIT * 1000

Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.

Разработка может считать безубыточной, если выручка не меньше стоимости разработки

In [19]:
min_product = BUDGET / PRODUCT_PROFIT / BEST_OIL_WELLS
min_product

111.11111111111111

Получилось 111.1, что выше, чем средние значения в каждом из регионов  
Таким образом, для покрытия расходов на разработку необходимы лучшие скважины

## Расчёт прибыли и рисков 

1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

Напишем фунцию для расчета прибыли

In [20]:
def profit(target, predictions, best_oil_wells, product_profit, budget):
    
    predictions_sorted = predictions.sort_values(ascending=False)
    
    best_predictions = target[predictions_sorted.index][:best_oil_wells]
    
    return (product_profit * best_predictions.sum() - (budget))

Создадим генератор случайных чисел

In [21]:
state = np.random.RandomState(12345)

Создадим функцию получения общей прибыли

In [22]:
def get_total_profit(common):
    total_profit = []
    for i in range(1000):
        subsample = common.sample(n=TOTAL_OIL_WELLS, replace=True, random_state=state)
    
        target_subsample = subsample['target']
    
        predictions_subsample = subsample['predictions'] 
    
        total_profit.append(profit(target_subsample, predictions_subsample, BEST_OIL_WELLS, PRODUCT_PROFIT, BUDGET))
    
    return total_profit

Создадим функцию получения значений средней прибыли, 95%-й доверительного интервал и риска убытков

In [23]:
def get_values(total_profit):
    lower = total_profit.quantile(0.025)
    upper = total_profit.quantile(0.975)
    mean = total_profit.mean()
    risks = (total_profit < 0).mean() * 100
    return {'lower':lower, 'upper':upper, 'mean':mean, 'risks':risks}

Создадим функцию вывода полученных значений на экран

In [24]:
def print_values(values):
    print('Средняя прибыль:', values.get('mean'))
    print('2.5%-квантиль', values.get('lower'))
    print('95%-ый доверительный интервал:', values.get('lower'), ':', values.get('upper'))
    print('Риски:', values.get('risks'), '\n')

Воспользуемся этими функциями для всех регионов

In [25]:
for common in common_list:
    total_profit = pd.Series(get_total_profit(common))
    values = get_values(total_profit)
    print_values(values)

Средняя прибыль: 425938526.91059244
2.5%-квантиль -102090094.83793654
95%-ый доверительный интервал: -102090094.83793654 : 947976353.3583689
Риски: 6.0 

Средняя прибыль: 518259493.6973249
2.5%-квантиль 128123231.43308444
95%-ый доверительный интервал: 128123231.43308444 : 953612982.0669085
Риски: 0.3 

Средняя прибыль: 420194005.3440501
2.5%-квантиль -115852609.16001143
95%-ый доверительный интервал: -115852609.16001143 : 989629939.8445739
Риски: 6.2 



Лучший результаты показывает второй регион, но не могу не обратить внимание, что у этого региона всего 12 значений в target  
* RMSE: 0.8931
* Средняя прибыль: 518 259 493
* 2.5%-квантиль 128 123 231
* 95%-ый доверительный интервал: 128 123 231 : 953 612 982
* Риски: 0.3

Из двух оставшихся результат лучше у первого региона, но в любом случае результаты не очень удовлетворительные и высокая среднеквадратичная ошибка